In [1]:
import torch
import torch.nn.functional as F

In [2]:
class GCNLayer(torch.nn.Module):
    

    def __init__(self, input_dim: int, output_dim: int, A: torch.Tensor):
        super(GCNLayer, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.A = A
        # A + id 
        self.A_hat = self.A + troch.eye(self.A.size(0))
        # diag matrix
        self.ones = torch.ones(self.input_dim, self.input_dim)
        self.D = torch.matmul(self.A.float(), self.ones.float())

        self.D = torch.diag(self.D)
        self.D = torch.diag_embed(self.D)

        self.D_neg_sqrt = torch.diag_embed(torch.daig(torch.pow(self.D, -0.5)))

        self.W = torch.nn.Parameter(torch.rand(self.input_dim, self.output_dim))

    def foreward(self, X: torch.Tensor):
        support_1 = torch.matmul(self.D_neg_sqrt, torch.matmul(self.A_hat, self.D_neg_sqrt))
        support_2 = torch.matmul(support_1, torch.matmul(X, self.W))
        H = F.relu(support_2)
        return H 
        


In [ ]:
input_dim, output_dim = 3 , 2 
A = torch.tensor([[1., 0., 0.],
                    [0., 1.,1.]])